## *Statistics coded:* Living conditions in Europe - income distribution and income inequality 

Prepared by [**Rodrigo González Santamaría**](rodrgo07@ucm.es) (EMOS).

This notebook aims at illustrating the *Statistics Explained* article on [Living conditions in Europe - income distribution and income inequality](https://ec.europa.eu/eurostat/statistics-explained/index.php?title=Living_conditions_in_Europe_-_income_distribution_and_income_inequality).


### Settings

Put all libraries/packages/modules/etc... that are necessary so that you can run this notebook.

In [ ]:
from eurostatapiclient import EurostatAPIClient
import json
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import matplotlib.patches as mpatches
import matplotlib.pyplot as mplt
import matplotlib.dates as mdates
from matplotlib.ticker import FuncFormatter, MaxNLocator, IndexLocator
from shapely import geometry
import geopandas as gpd
import requests
import io, os, re, sys
import warnings
import copy, functools
import zipfile
import folium
from folium import plugins as foliump
import branca.colormap as bcm

legend_properties = {'weight':'bold','size':'12'}
countries_names = {'AT':'Austria', 'BE':'Belgium', 'BG':'Bulgaria', 'CY': 'Cyprus', 'CZ': 'Czechia', 'DE': 'Germany', 'DK':
            'Denmark', 'EE':'Estonia', 'EL': 'Greece', 'ES':'Spain', 'FI':'Finland', 'FR':'France', 'HR':'Croatia', 
            'HU':'Hungary', 'IE':'Ireland', 'IT':'Italy', 'LT':'Lithuania', 'LU':'Luxembourg', 'LV':'Latvia', 'MT':
            'Malta', 'NL':'Netherlands', 'PL':'Poland', 'PT':'Portugal', 'RO':'Romania', 'SE':'Sweden', 'SI':'Slovenia',
            'SK':'Slovakia', 'UK':'United Kingdom', 'EU28': 'EU-28'}
countries_abbr = ['AT','BE','BG','CY','CZ','DE','DK','EE','EL','ES','FI',
                  'FR','HR','HU','IE','IT','LT','LU','LV','NV','NL','PL','PT','RO','SE','SI','SK','UK','MT', 'EU28']

In [ ]:
#Set versions and formats, so far only the ones used here are availeable and call client
VERSION = 'v2.1'
FORMAT = 'json'
LANGUAGE = 'en'
client = EurostatAPIClient(VERSION, FORMAT, LANGUAGE)

In [ ]:
# We save the direc
_THISDIR_ = !pwd
print('Current working directory: %s' % _THISDIR_)

### General overview
- **Key Findings**
- **Income distribution**
- **Income inequality**


### Documentation

### Setting Map

In [ ]:
# Graph the shape of Europe
# We first select the projection that will be used in this graph
PROJ = 'EPSG:4326'

In [ ]:
# We can then integrate the NUTS data [https://nbviewer.jupyter.org/github/eurostat/mortality-viz/blob/master/02_mortality_european_regions.ipynb]
GISCO_DOMAIN = 'https://gisco-services.ec.europa.eu/distribution/v2/'
NUTS_DOMAIN = 'nuts'

NUTS_YEAR = 2021
NUTS_RES = 20 # 60
NUTS_PROJ = PROJ.split(':')[-1] 
NUTS_FMT = 'geojson' # 'topojson'

nuts_template = 'NUTS_RG_%sM_%s_%s_LEVL_%%s.%s' % (NUTS_RES, NUTS_YEAR, NUTS_PROJ, NUTS_FMT) 
nuts_url = '%s/%s/%s' %  (GISCO_DOMAIN, NUTS_DOMAIN, NUTS_FMT)

# https://gisco-services.ec.europa.eu/distribution/v2/nuts/geojson/NUTS_RG_20M_2021_3857_LEVL_1.geojson
#    'NUTS_RG_%sM_%s_3035_LEVL_%s.%s'

print("Generic URL for NUTS data: \033[1m%s/%s\033[0m (where '%%s' represents a level)" 
      % (nuts_url, nuts_template))

In [ ]:
# We download the different NUTS data (there are four levels, each one corresponds to a different administrative divition level)

LEVELS = [0,1,2,3]

nuts_files = dict.fromkeys(LEVELS)
nuts_ext = NUTS_FMT

for level in LEVELS:
    nuts_file = nuts_template % level
    nuts_source = '%s/%s' % (nuts_url,nuts_file)
    try : 
        dest = os.path.join(_THISDIR_[0], nuts_file)
        assert os.path.exists(dest)
    except:
        try:
            !wget -O $dest "$nuts_source"
            #!mkdir $nuts_dir
            #!unzip -u -d $nuts_dir $nuts_source        
        except:
            raise IOError("Error fetching the data...")
        else:
            print('Data loaded on disk in directory: \033[1m%s\033[0m' % dest)
    else:
        print('Data already loaded on disk in directory: \033[1m%s\033[0m' % dest)
    finally:
        nuts_files.update({level: nuts_file}) # dest

In [ ]:
# We 'store' one vector dataset per NUTS level (nuts_data)

DRIVERS = {'geojson': 'GeoJSON',
          'shp': 'ESRI Shapefile'}

nuts_data = dict.fromkeys(LEVELS)
[nuts_data.update({l: gpd.read_file(os.path.join(_THISDIR_[0], nuts_files[l]), 
                                    driver=DRIVERS[NUTS_FMT])}) 
 for l in LEVELS]

print("Geographical datasets: \033[1m%s\033[0m loaded" % list(nuts_files.values()))
print("Projection: \033[1m%s\033[0m" % nuts_data[LEVELS[0]].crs)

In [ ]:
# Map of NUTS data (sin visualizar datos)
level = LEVELS[0]

f, ax = mplt.subplots(1, figsize=(16, 16))
nuts_data[level].plot(ax=ax)
ax.axes.get_xaxis().set_visible(False); ax.axes.get_yaxis().set_visible(False)
ax.set_title("NUTS at level %s (projection 'EPSG:%s')" % (level,NUTS_PROJ)) 
f.tight_layout()
mplt.show()

In [ ]:
# We define a very coarse bounding box that is embedded in the NUTS bounding box (NUTSarea) so as to render the data in a smaller region
EUmask = geometry.Polygon([#w: -25.5, s:35, e:30.2, n:71.3 in EPSG:4326 system
                         (-25.5, 30.2),
                         (45.3,  30.2),
                         (45.3,  71.3),
                         (-25.5, 71.3)
])
EUclip = gpd.GeoDataFrame(index=[0], geometry=[EUmask], crs=PROJ)

NUTSarea = gpd.GeoDataFrame(index=[0], geometry=[nuts_data[0].unary_union], crs=PROJ)


f, ax = mplt.subplots(1, figsize=(16, 10))
EUclip.boundary.plot(ax=ax, color='r', label='clip area')
NUTSarea.boundary.plot(ax=ax, color='b', label='NUTS area')
ax.axes.get_xaxis().set_visible(False); ax.axes.get_yaxis().set_visible(False)
ax.set_title('Clipped domain of representation')
ax.legend()
mplt.show()

## Key findings

*Ibid*, the code here will help reproducing the material presented in the section [Key findings](https://ec.europa.eu/eurostat/statistics-explained/index.php?title=Living_conditions_in_Europe_-_poverty_and_social_exclusion#Key_findings).

## Income distribution

*Ibid*, the code here will help reproducing the material presented in the section [Income distribution](https://ec.europa.eu/eurostat/statistics-explained/index.php?title=Living_conditions_in_Europe_-_income_distribution_and_income_inequality#Income_distribution).

### **MAP 1: Median equivalised disposable income in the EU-27**

In [ ]:
# We define query of dataframes from Eurostat query builder

dataset_map1_query = 'ilc_di03?precision=1&sex=T&indic_il=MED_E&geo=AT&geo=BE&geo=BG&geo=CH&geo=CY&geo=CZ&geo=DE&geo=DK&geo=EE&geo=EL&geo=ES&geo=EU27_2020&geo=FI&geo=FR&geo=HR&geo=HU&geo=IE&geo=IS&geo=IT&geo=LT&geo=LU&geo=LV&geo=ME&geo=MK&geo=MT&geo=NL&geo=NO&geo=PL&geo=PT&geo=RO&geo=RS&geo=SE&geo=SI&geo=SK&geo=TR&geo=UK&unit=PPS&time=2018&age=TOTAL'

# We use this query to retriev the data from the API Client
dataset_map1 = client.get_dataset(dataset_map1_query).to_dataframe().sort_values(by = ['values'],ascending = False)

# Simplify the dataset and change 
dataset_map1 = dataset_map1[['geo','values']]
dataset_map1.rename(columns=lambda x: x.strip(), inplace=True)

dataset_map1.head(n=5)

In [ ]:
# We add a conditional variable to set the colors of the map
trace = []
colors = []
for value in dataset_map1['values']:
    if value< 10000:
        colors.append('papayawhip')
    elif value< 15000:
        colors.append('bisque')
    elif value< 20000:
        colors.append('sandybrown')
    elif value< 25000:
        colors.append('orange')
    elif value >25000:
        colors.append('orangered')
    else:
        colors.append('darkgrey')
dataset_map1['colors'] = colors

dataset_map1.head(n = 5)

In [ ]:
# Check which data/countries (DATA_NUTS_ID) are actually made available throughout the dataset
KEY = 'NUTS_ID'

NUTS_ID = dict.fromkeys(LEVELS)
CTRY_ID = dict.fromkeys(LEVELS) 

for l in LEVELS:
    NUTS_ID.update({l: nuts_data[l][KEY].unique().tolist()})
    CTRY_ID.update({l: list(set([_id[:2] for _id in NUTS_ID[l]]))})
    
DATA_NUTS_ID = dict.fromkeys(LEVELS) 
all_id = dataset_map1['geo'].unique().tolist()

print('Data are availalble:') 
for l in LEVELS:
    DATA_NUTS_ID.update({l: list(set([_id[:2] for _id in all_id if len(_id)==l+2]))})
    print('* NUTS level %s: \033[1m%s\033[0m' 
          % (l, DATA_NUTS_ID[l]))

In [ ]:
# Check which country/datasets are missing
MISSING_CTRY_ID = dict.fromkeys(LEVELS) 

print('Data are NOT availalble:') 
for l in LEVELS:
    MISSING_CTRY_ID.update({l: list(set(CTRY_ID[l]).difference(set(DATA_NUTS_ID[l])))})
    print('* NUTS level %s: \033[1m%s\033[0m' 
          % (l, MISSING_CTRY_ID[l]))

In [ ]:
# Prepare dataframe to merge the NUTS DATA
dataset_map1.rename(columns = {'geo':'NUTS_ID'}, inplace = True)
dataset_map1.head(n = 5)

In [ ]:
# We merge NUTS_Data and Dataset_map1
country_data = nuts_data[LEVELS[0]]

# Merge on NUTS_ID
income_nuts = pd.merge(country_data, dataset_map1, how = 'inner', on = 'NUTS_ID') 
income_nuts.head()

colors = income_nuts['colors']

In [ ]:
LEVEL = 0
crop = gpd.clip(income_nuts, EUclip)
# crop = nuts_death_data[LEVEL].intersection(EUclip.unary_union) 

f, ax = mplt.subplots(1, figsize=(25, 20))
crop.plot(color=colors, ax=ax,  legend = True,
          legend_kwds = {'shrink':0.55},
          missing_kwds={ "color": "lightgrey", "alpha": 0.2, "edgecolor": "grey", "hatch": "///"}
         )

crop.boundary.plot(ax = ax, color = 'black')
ax.set_axis_off()
ax.set_title('Median equivalised disposable income, 2018', fontsize=30,fontweight='bold', horizontalalignment = 'cent', x = 0, y =1.05)
# f.tight_layout()
#f.subplots_adjust(hspace=0,wspace=-1)

# f.tight_layout()
#f.subplots_adjust(hspace=0,wspace=-1)

#Create custom legend

papayawhip_patch = mpatches.Patch(color='papayawhip', label='<10000')
bisque_patch = mpatches.Patch(color='bisque', label='>=10000 - <15000')
sandybrown_patch = mpatches.Patch(color='sandybrown', label='>=15000 - <20000')
orange_patch = mpatches.Patch(color='orange', label='>=20000 - <25000')
orangered_patch = mpatches.Patch(color='orangered', label='>=25000')

darkgrey_patch = mpatches.Patch(color='darkgrey', label='No data available')

leg= ax.legend(handles=[papayawhip_patch, sandybrown_patch, bisque_patch, orange_patch, orangered_patch, darkgrey_patch],frameon=False, loc=(0, -0.1), ncol=1, prop={'weight':'bold','size':'15'},labelspacing=0.5, handlelength=1)

### **Figure 1: Median equivalised disposable income of the population aged 18-64 years, analysed by educational level, 2018 (PPS per inhabitant)**

### Income inequality

*Ibid*, the code here will help reproducing the material presented in the section [Income inequality](https://ec.europa.eu/eurostat/statistics-explained/index.php?title=Living_conditions_in_Europe_-_income_distribution_and_income_inequality#Income_inequality).